In [ ]:
import asyncio
from datetime import datetime
import os

from astropy.time import Time
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Legend
from bokeh.plotting import figure
from IPython.display import Markdown as md
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tables as tb

from lsst_efd_client import EfdClient

DATETIME_FORMAT = '%Y%m%d_%H%M%S'
REPORT_TIME_FORMAT = '%F %T'

output_notebook(hide_banner=True)

In [ ]:
ipath = os.path.expanduser("~/Dropbox_LSST/Dome_Seeing_Monitor/DSM_Data/20200128")
date_str = "20200129_010709"
csc_index = 1

In [ ]:
ifname_centroid = os.path.join(ipath, f"smm_centroid_{date_str}.h5")

centroid_h5 = tb.open_file(ifname_centroid)
camera_info = centroid_h5.root.camera.info
general_info = centroid_h5.root.general.info

In [ ]:
md(f"### Dome Seeing Monitor Report {datetime.strptime(date_str, DATETIME_FORMAT).strftime(REPORT_TIME_FORMAT)}")

In [ ]:
print("General Information")
for name, value in zip(general_info.colnames, general_info[0]):
    print(f'{name}: {value.decode()}')
print()
print("Camera Information")
for name, value in zip(camera_info.colnames, camera_info[0]):
    try:
        v = value.decode()
    except AttributeError:
        v = value
    print(f'{name}: {v}')
print()
num_groups = len(list(centroid_h5.walk_nodes('/', 'Array'))) // 4
print(f"Number of data groups: {num_groups}")

In [ ]:
ifname_psd = os.path.join(ipath, f"smm_psd_{date_str}.h5")
# Get data keys from file
psd_h5 = tb.open_file(ifname_psd)
glist = psd_h5.root._f_list_nodes(classname="Group")
keys = [k._v_name for k in glist]

In [ ]:
x_data = []
y_data = []
frequencies = None
datetimes = []
timestamps = []
#print(keys)
for i, key in enumerate(keys):
    pd_h5 = pd.read_hdf(ifname_psd, key=key)
    x_data.append(pd_h5.X.values)
    y_data.append(pd_h5.Y.values)
    datetimes.append(datetime.strptime(key.replace('DT_', ''), DATETIME_FORMAT))
    timestamps.append(datetimes[-1].timestamp())
    if i == 0:
        frequencies = pd_h5.Frequencies.values

In [ ]:
x = np.stack(x_data)
y = np.stack(y_data)

In [ ]:
extent = (frequencies[0], frequencies[-1], timestamps[-1], timestamps[0])
xlabel = r'$\nu$ [1/s]'
plt.style.use('./plot_style.mplstyle')
fig = plt.figure(figsize=(14, 7))
fig.tight_layout()
ax1 = fig.add_subplot(121)
logNormX = LogNorm(vmin=np.min(x), vmax=np.max(x))
pos1 = ax1.imshow(x, aspect='auto', extent=extent, norm=logNormX)
date_fmt = [datetime.strftime(datetime.fromtimestamp(d), '%Y-%m-%d %H:%M:%S') for d in ax1.get_yticks()]
ax1.set_title('PSD X')
ax1.set_yticklabels(date_fmt)
ax1.set_xlabel(xlabel)
fig.colorbar(pos1, orientation='horizontal')

ax2 = fig.add_subplot(122)
logNormY = LogNorm(vmin=np.min(y), vmax=np.max(y))
pos2 = ax2.imshow(y, aspect='auto', extent=extent, norm=logNormY)
ax2.set_title('PSD Y')
ax2.set_yticks([])
plt.setp(ax2.get_yticklabels(), visible=False)
ax2.set_xlabel(xlabel)
fig.colorbar(pos2, orientation='horizontal')

fig.subplots_adjust(wspace=0.01)

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
start = Time(datetimes[0])
end = Time(datetimes[-1])
df = await client.select_time_series('lsst.sal.DSM.domeSeeing', '*', start.tai, end.tai, csc_index)
df_w = await client.select_time_series('lsst.sal.Environment.windSpeed', '*', start.tai, end.tai, csc_index)

In [ ]:
source = ColumnDataSource(df)
source_w = ColumnDataSource(df_w)
dt_format = '%F %T'
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
tick_rotation = 75. * np.pi / 180.

s1 = figure(title = "Centroids")
cx = s1.line(x='index', y='centroidX', source=source, line_color='blue', line_width=2)
cy = s1.line(x='index', y='centroidY', source=source, line_color='black', line_width=2)
s1.xaxis.formatter = DatetimeTickFormatter(**formats)
s1.xaxis.major_label_orientation = tick_rotation
legend1 = Legend(items=[
    ("X" , [cx]),
    ("Y" , [cy]),
], location="center")
s1.add_layout(legend1, 'right')
s1.legend.margin = 0
s1.legend.padding = 0
s2 = figure(title='Seeing')
sx = s2.line(x='index', y='rmsX', source=source, line_color='blue', line_width=2)
sy = s2.line(x='index', y='rmsY', source=source, line_color='black', line_width=2)
s2.xaxis.formatter = DatetimeTickFormatter(**formats)
s2.xaxis.major_label_orientation = tick_rotation
s2.yaxis.axis_label = 'scaled RMS (arcseconds)'
legend2 = Legend(items=[
    ("X" , [sx]),
    ("Y" , [sy]),
], location="center")
s2.add_layout(legend2, 'right')
s2.legend.margin = 0
s2.legend.padding = 0
s3 = figure(title='Wind')
s3.line(x='index', y='avg2M', source=df_w, line_width=2)
s3.xaxis.formatter = DatetimeTickFormatter(**formats)
s3.xaxis.major_label_orientation = tick_rotation
s3.yaxis.axis_label = 'speed (m/s)'
s4 = figure(title='Flux')
s4.line(x='index', y='flux', source=df, line_width=2)
s4.xaxis.formatter = DatetimeTickFormatter(**formats)
s4.xaxis.major_label_orientation = tick_rotation
s5 = figure(title='Max ADC')
s5.line(x='index', y='maxADC', source=df, line_width=2)
s5.xaxis.formatter = DatetimeTickFormatter(**formats)
s5.xaxis.major_label_orientation = tick_rotation
s6 = figure(title='FWHM')
s6.line(x='index', y='fwhm', source=df, line_width=2)
s6.xaxis.formatter = DatetimeTickFormatter(**formats)
s6.xaxis.major_label_orientation = tick_rotation
s6.yaxis.axis_label = '(pixels)'
grid = gridplot([[s1, s2, s3], [s4, s5, s6]], plot_width=300, plot_height=300)
s1.x_range = s2.x_range = s3.x_range = s4.x_range = s5.x_range = s6.x_range
show(grid)